# Benchmark: Array Loading Performance

Tests MboRawArray initialization, single/multi-frame access, phase correction methods, and reduction operations.

**Reference Dataset:** `\\rbo-s1\S1_DATA\lbm\kbarber\2025-11-04-mk311\raw\green`

In [ ]:
from datetime import date
import mbo_utilities
import platform
import os

BENCHMARK_DATE = date.today().isoformat()

print(f"Benchmark Date: {BENCHMARK_DATE}")
print(f"mbo_utilities: {mbo_utilities.__version__}")
print()

print("=" * 60)
print("SYSTEM INFO")
print("=" * 60)
print(f"OS: {platform.system()} {platform.release()}")
print(f"Python: {platform.python_version()}")
print(f"CPU: {platform.processor()}")
print(f"CPU cores: {os.cpu_count()}")

try:
    import psutil
    mem = psutil.virtual_memory()
    print(f"RAM: {mem.total / 1024**3:.1f} GB total, {mem.available / 1024**3:.1f} GB available ({mem.percent}% used)")
    disk = psutil.disk_usage('C:/')
    print(f"Disk (C:): {disk.total / 1024**3:.0f} GB total, {disk.free / 1024**3:.0f} GB free")
except ImportError:
    print("(install psutil for memory/disk info)")

import numpy as np
print(f"NumPy: {np.__version__}")
print("=" * 60)

In [ ]:
from pathlib import Path
import time
import numpy as np
from mbo_utilities.arrays.tiff import MboRawArray

# reference dataset
TEST_PATH = r"\\rbo-s1\S1_DATA\lbm\kbarber\2025-11-04-mk311\raw\green"

p = Path(TEST_PATH)
files = sorted(list(p.glob("*.tif")) + list(p.glob("*.tiff")))
print(f"Found {len(files)} files")

## 1. Array Initialization

In [ ]:
%%timeit -n 1 -r 3 -o -q
arr = MboRawArray(files=files)

In [ ]:
init_time = _.average
print(f"Array init: {init_time*1000:.0f} ms")

arr = MboRawArray(files=files)
print(f"Shape: {arr.shape}")
print(f"Dtype: {arr.dtype}")
print(f"Z-planes: {arr.num_channels}")

## 2. Single Frame Access (no phase correction)

In [ ]:
arr.fix_phase = False

# single frame, single z-plane
times = []
for i in range(20):
    t0 = time.perf_counter()
    frame = arr[i % arr.num_frames, 0]
    t1 = time.perf_counter()
    times.append((t1 - t0) * 1000)

single_frame_nophase = np.mean(times)
print(f"arr[i, 0] (single frame, single z):")
print(f"  Mean: {single_frame_nophase:.2f} ms")
print(f"  Min:  {np.min(times):.2f} ms")
print(f"  Max:  {np.max(times):.2f} ms")

In [ ]:
# single frame, ALL z-planes
times = []
for i in range(20):
    t0 = time.perf_counter()
    frame = arr[i % arr.num_frames]
    t1 = time.perf_counter()
    times.append((t1 - t0) * 1000)

single_frame_allz = np.mean(times)
print(f"arr[i] (single frame, ALL z-planes):")
print(f"  Mean: {single_frame_allz:.2f} ms")
print(f"  Min:  {np.min(times):.2f} ms")
print(f"  Max:  {np.max(times):.2f} ms")

## 3. Single Frame Access (with phase correction)

In [ ]:
arr.fix_phase = True
arr.use_fft = False  # integer method

times = []
for i in range(20):
    t0 = time.perf_counter()
    frame = arr[i % arr.num_frames, 0]
    t1 = time.perf_counter()
    times.append((t1 - t0) * 1000)

phase_int = np.mean(times)
print(f"arr[i, 0] WITH phase corr (integer method):")
print(f"  Mean: {phase_int:.2f} ms")
print(f"  Min:  {np.min(times):.2f} ms")
print(f"  Max:  {np.max(times):.2f} ms")

In [ ]:
arr.use_fft = True  # FFT method

times = []
for i in range(20):
    t0 = time.perf_counter()
    frame = arr[i % arr.num_frames, 0]
    t1 = time.perf_counter()
    times.append((t1 - t0) * 1000)

phase_fft = np.mean(times)
print(f"arr[i, 0] WITH phase corr (FFT method):")
print(f"  Mean: {phase_fft:.2f} ms")
print(f"  Min:  {np.min(times):.2f} ms")
print(f"  Max:  {np.max(times):.2f} ms")

## 4. Multi-Frame Access

In [ ]:
arr.fix_phase = False
multi_frame_results = {}

for n in [10, 50, 100, 500]:
    if n > arr.num_frames:
        continue
    times = []
    for _ in range(3):
        t0 = time.perf_counter()
        data = arr[:n, 0]
        t1 = time.perf_counter()
        times.append((t1 - t0) * 1000)
    multi_frame_results[n] = np.mean(times)
    print(f"arr[:{n}, 0]: {np.mean(times):.1f} ms (shape={data.shape})")

## 5. Reduction Operations

In [ ]:
arr.fix_phase = False
n_frames = min(50, arr.num_frames)
print(f"Testing reductions on first {n_frames} frames, z-plane 0")

reduction_results = {}

# mean
t0 = time.perf_counter()
result = arr[:n_frames, 0].mean(axis=0)
t1 = time.perf_counter()
reduction_results['mean'] = (t1-t0)*1000
print(f"mean(axis=0): {(t1-t0)*1000:.1f} ms")

# max projection
t0 = time.perf_counter()
result = arr[:n_frames, 0].max(axis=0)
t1 = time.perf_counter()
reduction_results['max'] = (t1-t0)*1000
print(f"max(axis=0): {(t1-t0)*1000:.1f} ms")

# std
t0 = time.perf_counter()
result = arr[:n_frames, 0].std(axis=0)
t1 = time.perf_counter()
reduction_results['std'] = (t1-t0)*1000
print(f"std(axis=0): {(t1-t0)*1000:.1f} ms")

## Summary

In [ ]:
print("=" * 70)
print("LOADING BENCHMARK SUMMARY")
print("=" * 70)
print(f"Date: {BENCHMARK_DATE}")
print(f"mbo_utilities: {mbo_utilities.__version__}")
print(f"Data: {TEST_PATH}")
print(f"Shape: {arr.shape}")
print(f"Files: {len(files)}")
print()
print(f"{'Metric':<40} {'Time':>12}")
print("-" * 55)
print(f"{'Array init':<40} {init_time*1000:>10.0f} ms")
print(f"{'Single frame (no phase)':<40} {single_frame_nophase:>10.2f} ms")
print(f"{'Single frame (all z, no phase)':<40} {single_frame_allz:>10.2f} ms")
print(f"{'Single frame (phase int)':<40} {phase_int:>10.2f} ms")
print(f"{'Single frame (phase FFT)':<40} {phase_fft:>10.2f} ms")
for n, t in multi_frame_results.items():
    print(f"{f'Multi-frame ({n} frames)':<40} {t:>10.1f} ms")
for op, t in reduction_results.items():
    print(f"{f'Reduction {op} (50 frames)':<40} {t:>10.1f} ms")
print("=" * 70)

# cleanup
for tf in arr.tiff_files:
    tf.close()